In [ ]:
import numpy as np
import pandas as pd

# =========================
# 0) 入力・列名設定
# =========================
INPUT_PATH = "/Users/ayo/Desktop/_GSAIS_/Research/OralHealth_tokyo/paper_analysis/data/AllData_tillMar2024.csv"   # .csv でもOK

ID_COL  = "No_All"
AGE_COL = "age"
SEX_COL = "sex"   # "男/女" or "M/F" など


# 永久歯・乳歯の列（あなた指定）
perm_cols = [
    'U17', 'U16', 'U15', 'U14', 'U13', 'U12', 'U11', 'U21', 'U22', 'U23', 'U24', 'U25', 'U26', 'U27',
    'L37', 'L36', 'L35', 'L34', 'L33', 'L32', 'L31', 'L41', 'L42', 'L43', 'L44', 'L45', 'L46', 'L47'
]
baby_cols = [
    'u55', 'u54', 'u53', 'u52', 'u51', 'u61', 'u62', 'u63', 'u64', 'u65',
    'l75', 'l74', 'l73', 'l72', 'l71', 'l81', 'l82', 'l83', 'l84', 'l85'
]

# =========================
# 1) あなたの歯コード定義（固定）
# =========================
CODE_TREATED = 1   # 処置
CODE_C0      = 2   # 要観察歯
CODE_CARIES  = 3   # C（未処置）
CODE_MISSING = 4   # 喪失
CODE_TRAUMA  = 7   # 外傷
CODE_RDT     = 8   # 残根（未処置扱い）

# D/d には 3 と 8 を含める（あなたの定義）
DECAY_SET = {CODE_CARIES, CODE_RDT}

# =========================
# 2) 年齢範囲と年齢階級
# =========================
AGE_MIN, AGE_MAX = 2, 17
AGE_BINS   = [2, 6, 12, 18]                 # [2-5], [6-11], [12-17]
AGE_LABELS = ["2-5", "6-11", "12-17"]


# =========================
# Utility
# =========================
def read_input(path, sheet=None):
    if path.lower().endswith(".csv"):
        return pd.read_csv(path)
    return pd.read_excel(path, sheet_name=sheet)

def ensure_cols_exist(df, cols, label):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"[{label}] columns not found: {missing}")

def standardize_sex(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {"m", "male", "男", "1"}:
        return "Male"
    if s in {"f", "female", "女", "2"}:
        return "Female"
    return str(x)

def to_numeric_teeth(df, cols):
    # 歯コードを数値化（文字の"3"等も吸収）
    out = df.copy()
    out[cols] = out[cols].apply(pd.to_numeric, errors="coerce")
    return out

def safe_mean_sd(series):
    x = pd.to_numeric(series, errors="coerce")
    return pd.Series({"mean": x.mean(skipna=True), "sd": x.std(skipna=True)})

def add_age_groups(df):
    df = df.copy()
    df[AGE_COL] = pd.to_numeric(df[AGE_COL], errors="coerce")
    df = df[(df[AGE_COL] >= AGE_MIN) & (df[AGE_COL] <= AGE_MAX)].copy()
    df["age_single"] = df[AGE_COL].round().astype("Int64")
    df["age_group"] = pd.cut(df[AGE_COL], bins=AGE_BINS, right=False, labels=AGE_LABELS)
    return df

def classify_df_treatment(d, f):
    # df>0の人の処置状況（実態調査系）
    if d == 0 and f > 0:
        return "Completely treated"
    if d > 0 and f > 0:
        return "Partially treated"
    if d > 0 and f == 0:
        return "Untreated"
    return "No df"

def make_n_pct(tab, n_col="n"):
    out = tab.copy()
    for c in out.columns:
        if c == n_col:
            continue
        if pd.api.types.is_numeric_dtype(out[c]):
            out[f"{c}_pct"] = out[c] / out[n_col] * 100
    return out

def per_row_count(df, cols, code_or_set):
    sub = df[cols]
    if isinstance(code_or_set, set):
        return sub.isin(code_or_set).sum(axis=1)
    else:
        return (sub == code_or_set).sum(axis=1)


# =========================
# 個人指標の作成（あなたの定義に準拠）
# =========================
def build_individual_indices(df):
    df = df.copy()

    # 永久歯
    df["Perm_D"] = per_row_count(df, perm_cols, DECAY_SET)        # 3 or 8
    df["Perm_M"] = per_row_count(df, perm_cols, CODE_MISSING)     # 4
    df["Perm_F"] = per_row_count(df, perm_cols, CODE_TREATED)     # 1
    df["Perm_DMFT"] = df["Perm_D"] + df["Perm_M"] + df["Perm_F"]

    df["Perm_C0"] = per_row_count(df, perm_cols, CODE_C0)         # 2
    df["Perm_DMFT_C0"] = df["Perm_DMFT"] + df["Perm_C0"]

    # 乳歯
    df["Baby_d"] = per_row_count(df, baby_cols, DECAY_SET)        # 3 or 8
    df["Baby_m"] = per_row_count(df, baby_cols, CODE_MISSING)     # 4
    df["Baby_f"] = per_row_count(df, baby_cols, CODE_TREATED)     # 1
    # ユーザー定義では Baby_DMFT という名前で d+m+f を作っているので合わせる
    df["Baby_DMFT"] = df["Baby_d"] + df["Baby_m"] + df["Baby_f"]

    df["Baby_C0"] = per_row_count(df, baby_cols, CODE_C0)         # 2
    df["Baby_DMFT_C0"] = df["Baby_DMFT"] + df["Baby_C0"]

    # 全歯（永久+乳歯）での追加カウント
    all_cols = perm_cols + baby_cols
    df["count_C0_all"] = per_row_count(df, all_cols, CODE_C0)         # 2
    df["count_Trauma_all"] = per_row_count(df, all_cols, CODE_TRAUMA) # 7
    df["count_RDT_all"] = per_row_count(df, all_cols, CODE_RDT)       # 8

    # 参考：う蝕経験の有無
    df["Perm_DMF_any"] = df["Perm_DMFT"] > 0
    df["Perm_D_any"]   = df["Perm_D"] > 0
    df["Baby_df_any"]  = (df["Baby_d"] + df["Baby_f"]) > 0   # 乳歯は df（m除外）を処置状況に使うことが多い
    df["Baby_d_any"]   = df["Baby_d"] > 0

    # 乳歯の「処置状況」（df>0対象）
    df["Baby_df_status"] = df.apply(lambda r: classify_df_treatment(int(r["Baby_d"]), int(r["Baby_f"])), axis=1)

    return df


# =========================
# 集計表（実態調査風）
# =========================
def summarize_counts_and_rates(df, group_cols):
    # 被調査者数
    n = df.groupby(group_cols, dropna=False).size().rename("n").reset_index()

    # 永久歯：DMFあり/なし、（参考）Dあり
    perm_any = (df.groupby(group_cols, dropna=False)["Perm_DMF_any"]
                  .agg(no_DMF=lambda x: (~x).sum(), with_DMF=lambda x: x.sum())
                  .reset_index())
    perm_refD = (df.groupby(group_cols, dropna=False)["Perm_D_any"]
                   .sum().rename("with_D")
                   .reset_index())

    tab_perm = n.merge(perm_any, on=group_cols, how="left").merge(perm_refD, on=group_cols, how="left").fillna(0)
    tab_perm = make_n_pct(tab_perm, n_col="n")

    # 乳歯：dfあり/なし、処置状況（df>0のみ）
    baby_any = (df.groupby(group_cols, dropna=False)["Baby_df_any"]
                  .agg(no_df=lambda x: (~x).sum(), with_df=lambda x: x.sum())
                  .reset_index())
    baby_status = (df[df["Baby_df_any"]]
                   .groupby(group_cols, dropna=False)["Baby_df_status"]
                   .value_counts()
                   .unstack(fill_value=0)
                   .reset_index())

    tab_baby = n.merge(baby_any, on=group_cols, how="left").merge(baby_status, on=group_cols, how="left").fillna(0)
    tab_baby = make_n_pct(tab_baby, n_col="n")

    return tab_perm, tab_baby, n

def summarize_mean_sd(df, group_cols):
    # 平均（SD）—永久歯
    perm_vars = ["Perm_D","Perm_M","Perm_F","Perm_DMFT","Perm_C0","Perm_DMFT_C0"]
    baby_vars = ["Baby_d","Baby_m","Baby_f","Baby_DMFT","Baby_C0","Baby_DMFT_C0"]
    other_vars = ["count_C0_all","count_Trauma_all","count_RDT_all"]

    perm = (df.groupby(group_cols, dropna=False)[perm_vars]
            .agg(["mean","std"]).reset_index())
    perm.columns = ["_".join([c for c in col if c]) if isinstance(col, tuple) else col for col in perm.columns]

    baby = (df.groupby(group_cols, dropna=False)[baby_vars]
            .agg(["mean","std"]).reset_index())
    baby.columns = ["_".join([c for c in col if c]) if isinstance(col, tuple) else col for col in baby.columns]

    other = (df.groupby(group_cols, dropna=False)[other_vars]
             .agg(["mean","std"]).reset_index())
    other.columns = ["_".join([c for c in col if c]) if isinstance(col, tuple) else col for col in other.columns]

    return perm, baby, other

def tooth_code_distribution(df, tooth_cols, group_cols, dataset_label):
    """
    表14系：歯別×コードの分布（コード=1/2/3/4/7/8/その他）
    """
    tmp = df[[ID_COL] + group_cols + tooth_cols].copy()
    long = tmp.melt(id_vars=[ID_COL] + group_cols, var_name="tooth", value_name="code")
    # 欠損も含めた分布が欲しい場合は dropna=False を活用
    dist = (long.groupby(group_cols + ["tooth", "code"], dropna=False)
            .size().rename("count").reset_index())
    dist["dataset"] = dataset_label
    return dist

def tooth_category_distribution(df, tooth_cols, group_cols, dataset_label):
    """
    表14を「コード」ではなく「カテゴリ（Treated/C0/Decayed/Missing/Trauma/Other/NA）」で出す版
    """
    tmp = df[[ID_COL] + group_cols + tooth_cols].copy()
    long = tmp.melt(id_vars=[ID_COL] + group_cols, var_name="tooth", value_name="code")

    def cat(x):
        if pd.isna(x):
            return "NA"
        if x == CODE_TREATED:
            return "Treated(1)"
        if x == CODE_C0:
            return "C0(2)"
        if x in DECAY_SET:
            # 3 or 8 を分けたいならここを拡張
            return "Decayed_or_RDT(3/8)"
        if x == CODE_MISSING:
            return "Missing(4)"
        if x == CODE_TRAUMA:
            return "Trauma(7)"
        return "Other"

    long["category"] = long["code"].apply(cat)
    dist = (long.groupby(group_cols + ["tooth", "category"], dropna=False)
            .size().rename("count").reset_index())
    dist["dataset"] = dataset_label
    return dist


# =========================
# Main
# =========================
def main():
    df = read_input(INPUT_PATH, SHEET_NAME)
    df = df.copy()

    # 列チェック
    for c in [ID_COL, AGE_COL, SEX_COL]:
        if c not in df.columns:
            raise ValueError(f"Required column not found: {c}")
    ensure_cols_exist(df, perm_cols, "perm_cols")
    ensure_cols_exist(df, baby_cols, "baby_cols")

    # 前処理：性の標準化、歯コードの数値化、年齢範囲/階級
    df[SEX_COL] = df[SEX_COL].apply(standardize_sex)
    df = to_numeric_teeth(df, perm_cols + baby_cols)
    df = add_age_groups(df)

    # 個人指標を付与
    df = build_individual_indices(df)

    # グループ（実態調査っぽく：性×年齢階級 と 性×単歳）
    grp_agegroup = [SEX_COL, "age_group"]
    grp_single   = [SEX_COL, "age_single"]

    # 人数・割合・処置状況
    tab_perm_agegroup, tab_baby_agegroup, n_agegroup = summarize_counts_and_rates(df, grp_agegroup)
    tab_perm_single,   tab_baby_single,   n_single   = summarize_counts_and_rates(df, grp_single)

    # 平均・SD
    mean_perm_agegroup, mean_baby_agegroup, mean_other_agegroup = summarize_mean_sd(df, grp_agegroup)
    mean_perm_single,   mean_baby_single,   mean_other_single   = summarize_mean_sd(df, grp_single)

    # 歯別分布（コード版とカテゴリ版）
    dist_perm_code = tooth_code_distribution(df, perm_cols, grp_agegroup, "permanent")
    dist_baby_code = tooth_code_distribution(df, baby_cols, grp_agegroup, "baby")
    dist_perm_cat  = tooth_category_distribution(df, perm_cols, grp_agegroup, "permanent")
    dist_baby_cat  = tooth_category_distribution(df, baby_cols, grp_agegroup, "baby")

    # 出力
    out_path = "output_tables.xlsx"
    with pd.ExcelWriter(out_path, engine="openpyxl") as w:
        # 被調査者数
        n_agegroup.to_excel(w, sheet_name="1_n_agegroup", index=False)
        n_single.to_excel(w,   sheet_name="1_n_single",   index=False)

        # 永久歯：DMF有無（表10系）
        tab_perm_agegroup.to_excel(w, sheet_name="10_perm_DMF_any_agegroup", index=False)
        tab_perm_single.to_excel(w,   sheet_name="10_perm_DMF_any_single",   index=False)

        # 乳歯：df有無＋処置状況（表8系）
        tab_baby_agegroup.to_excel(w, sheet_name="8_baby_df_status_agegroup", index=False)
        tab_baby_single.to_excel(w,   sheet_name="8_baby_df_status_single",   index=False)

        # 平均・SD（表13系＋C0込み指標）
        mean_perm_agegroup.to_excel(w, sheet_name="13_perm_mean_sd_agegroup", index=False)
        mean_baby_agegroup.to_excel(w, sheet_name="13_baby_mean_sd_agegroup", index=False)
        mean_other_agegroup.to_excel(w, sheet_name="13_other_mean_sd_agegroup", index=False)

        mean_perm_single.to_excel(w, sheet_name="13_perm_mean_sd_single", index=False)
        mean_baby_single.to_excel(w, sheet_name="13_baby_mean_sd_single", index=False)
        mean_other_single.to_excel(w, sheet_name="13_other_mean_sd_single", index=False)

        # 歯別分布（表14系）
        dist_perm_code.to_excel(w, sheet_name="14_perm_tooth_code_dist", index=False)
        dist_baby_code.to_excel(w, sheet_name="14_baby_tooth_code_dist", index=False)
        dist_perm_cat.to_excel(w,  sheet_name="14_perm_tooth_cat_dist", index=False)
        dist_baby_cat.to_excel(w,  sheet_name="14_baby_tooth_cat_dist", index=False)

        # 個票（確認用：必要ならコメントアウト）
        df.to_excel(w, sheet_name="0_individual_with_indices", index=False)

    print(f"Saved: {out_path}")

if __name__ == "__main__":
    main()



/var/folders/nc/3dnb813d3ld83hg56j5wbw2w0000gn/T/ipykernel_35536/1347871611.py:162: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  n = df.groupby(group_cols, dropna=False).size().rename("n").reset_index()
/var/folders/nc/3dnb813d3ld83hg56j5wbw2w0000gn/T/ipykernel_35536/1347871611.py:165: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  perm_any = (df.groupby(group_cols, dropna=False)["Perm_DMF_any"]
/var/folders/nc/3dnb813d3ld83hg56j5wbw2w0000gn/T/ipykernel_35536/1347871611.py:168: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=F

TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first